## Detailed Article Explaination

The detailed code explanation for this article is available at the following link:

https://www.daniweb.com/programming/computer-science/tutorials/541682/paris-olympics-chatbot-get-ticket-information-using-chat-gpt-and-langchain

For my other articles for Daniweb.com, please see this link:

https://www.daniweb.com/members/1235222/usmanmalik57

## Installing and Importing Required Libraries

In [34]:
!pip install -U langchain
!pip install langchain-openai
!pip install pypdf
!pip install faiss-cpu

   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/14.5 MB 656.4 kB/s eta 0:00:22
   ---------------------------------------- 0.1/14.5 MB 944.1 kB/s eta 0:00:16
   ---------------------------------------- 0.2/14.5 MB 871.5 kB/s eta 0:00:17
    --------------------------------------- 0.2/14.5 MB 958.6 kB/s eta 0:00:15
   - -------------------------------------- 0.4/14.5 MB 1.2 MB/s eta 0:00:12
   - -------------------------------------- 0.4/14.5 MB 1.2 MB/s eta 0:00:12
   - -------------------------------------- 0.5/14.5 MB 1.2 MB/s eta 0:00:12
   - -------------------------------------- 0.5/14.5 MB 1.2 MB/s eta 0:00:12
   - -------------------------------------- 0.6/14.5 MB 1.1 MB/s eta 0:00:13
   - -------------------------------------- 0.6/14.5 MB 1.2 MB/s eta 0:00:12
   - -------

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.documents import Document
import os

## Generate Default Responses from Chat-GPT

In [37]:
openai_key = os.environ.get('OPENAI_KEY2')

In [38]:
llm = ChatOpenAI(
    openai_api_key = openai_key ,
    model = 'gpt-4',
    temperature = 0.5
)

In [39]:
result = llm.invoke("You are a comedian, tell a Joke about maths.")
print(result.content)

Why was the math book sad?

Because it had too many problems!


In [5]:
prompt = ChatPromptTemplate.from_messages([
    ("system", '{assistant}'),
    ("user", "{input}")
])

In [6]:
chain = prompt | llm 

result = chain.invoke(
    {"assistant": "You are a comedian",
     "input": "Tell a joke about mathematics"}
)

print(result.content)

Why was the math book sad?

Because it had too many problems!


In [7]:
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

result = chain.invoke(
    {"assistant": "You are a comedian",
     "input": "Tell a joke about mathematics"}
)
print(result)

Why was the math book sad?

Because it had too many problems!


In [8]:
result = chain.invoke(
    {"assistant": "You are a ticket receptionist",
     "input": "What is the lowest ticket price for tennis games in Paris Olympis 2024?"})
print(result)

I'm sorry, as an AI developed by OpenAI, I don't have real-time data or the ability to browse the internet. Therefore, I can't provide the current ticket prices for specific events such as the Paris Olympics 2024. I would recommend checking the official Olympics website or authorized ticket sellers for the most accurate and up-to-date information.


## Augmented Chat-GPT Responses with RAG Using LangChain

### Splitting and Embedding Ticket Price Information Document

In [4]:
loader = PyPDFLoader("https://tickets.paris2024.org/obj/media/FR-Paris2024/ticket-prices.pdf")
docs = loader.load_and_split()

In [45]:
embeddings = OpenAIEmbeddings(openai_api_key = openai_key)

In [87]:
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

### Question Answering with Chatbot

In [92]:
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

Question: {input}

Context: {context}
"""
)

document_chain = create_stuff_documents_chain(llm, prompt)

In [91]:


document_chain.invoke({
    "input": "What is the lowest ticket price for tennis games in Paris Olympis 2024?",
    "context": [Document(page_content = "The ticket prices for tennis games are 15, 10, 25 euros")]
})

'The lowest ticket price for tennis games in Paris Olympis 2024 is 10 euros.'

In [94]:
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [99]:
def generate_response(query):
    response = retrieval_chain.invoke({"input": query})
    print(response["answer"])

In [103]:
query = "What is the lowest ticket price for tennis games?"
generate_response(query)


The lowest ticket price for tennis games is €30.


In [104]:
query = "What is the category for the lowest ticket price for tennis games?"
generate_response(query)

The category for the lowest ticket price for tennis games is D.


In [109]:
query = "What is maximum price for category B for men's single tennis games for non-medal games?"
generate_response(query)

The maximum price for category B for men's single tennis games for non-medal games is €185.
